<h1 style="color: rgba(237, 153, 29, 1);">Aula Prática - Problemas Clássicos de Coordenação</h1>
<h2>Produtores e Consumidores</h2>

<small>
<p><strong>IMPORTANTE</strong>: O comando '%%file' é usado no Python para criar arquivos .java no diretório onde este notebook está salvo. Os arquivos criados são nomeados conforme o identificador fornecido após o comando '%%file'.</p>
</small>

<h3>O Problema</h3>

<p>O problema dos produtores e consumidores (também chamado de problema do buffer limitado) é um problema clássico de sincronização em sistemas concorrentes.</p>

<p>Ele descreve uma situação em que:</p>

<ul>
    <li><strong style="color: #E0901B;">Produtores</strong>: processos ou threads que geram dados (itens) e os colocam em um buffer compartilhado;</li>
    <li><strong style="color: #E0901B;">Consumidores</strong>: são processos ou threads que retiram dados do buffer para processá-los.</li>
</ul>

<h4>Restrições</h4>

<ol>
    <li> O buffer tem capacidade limitada (não pode crescer indefinidamente);</li>
    <li> Os produtores não podem inserir novos itens se o buffer já estiver cheio;</li>
    <li> Os consumidores não podem retirar itens se o buffer estiver vazio;</li>
    <li> É necessário garantir exclusão mútua (mutex) para que dois processos não tentem acessar/modificar o buffer ao mesmo tempo, evitando inconsistências.</li>
</ol>

<h4>Mecanismos de Coordenação</h4>

<p>O funcionamento correto do sistema depende de semáforos para coordenar produtores e consumidores (<strong>Figura 1</strong>):</p>

<ul>
    <li><strong style="color: #E0901B;">semáforo para vagas(N)</strong>: O número de espaços livres (<strong>N</strong>) no buffer;</li>
    <li><strong style="color: #E0901B;">semáforo para itens(0)</strong>: O número de itens disponíveis no buffer para consumo (<strong>inicialmente zero</strong>);</li>
    <li><strong style="color: #E0901B;">mutex</strong>: Garantir a exclusão mútua ao acessar o um item do buffer.</li>
</ul>

<figure>
    <img src="imagem.png" alt="Mecanismos de coordenação usados no problema dos produtores e consumidores">
    <figcaption>Figura 1: Coordenação das ações de produtores e consumidores.</figcaption>
</figure>

<h3>Escrita de Arquivos em C</h3>

<p>A manipulação de arquivos na biblioteca padrão de C (stdio.h) funciona com base em um ponteiro especial: FILE*.</p>

<p>Para escrever em um arquivo, devemos abrir o arquivo em modo de escrita ("w"). Se o arquivo não existir, ele será criado. Se existir, seu conteúdo é apagado. O Código 1 (<a href="https://github.com/jjbaqueta/SO/blob/main/exemplos/aberturaArq.c">download do código</a>) apresenta a operação de abertura de um arquivo chamado "arquivo.txt".
</p>

<h4 style="color: #2d3436;"><strong>Código 1</strong>: Escrevendo em um arquivo de texto.</h4>

In [1]:
%%file aberturaArq.c

// Comando de compilação: gcc -Wall aberturaArq.c -o out
// Comando de execução: ./out

#include <stdio.h>

int main() {
    FILE* arquivo = fopen("arquivo.txt", "w");

    if (arquivo == NULL) {
        perror("Erro ao abrir o arquivo de entrada");
        return 1;
    }

    int a = 5;
    int b = 10;

    fprintf(arquivo, "%d %d\n", a, b);
    fclose(arquivo);

    return 0;
}

Writing aberturaArq.c


<h3>Leitura de Arquivos em C</h3>

<p>Para realizar a leitura, abrimos o arquivo em modo de leitura ("r"). Portanto, o programa precisa que o arquivo já exista. O Código 2 (<a href="https://github.com/jjbaqueta/SO/blob/main/exemplos/leituraArq.c">download do código</a>) apresenta a operação de leitura de um arquivo chamado "arquivo.txt".</p>

<h4 style="color: #2d3436;"><strong>Código 2</strong>: Lendo um arquivo de texto.</h4>

In [4]:
%%file leituraArq.c

// Comando de compilação: gcc -Wall leituraArq.c -o out
// Comando de execução: ./out

#include <stdio.h>

#define TAMANHO_LINHA 50

int main() {
    FILE* arquivo = fopen("arquivo.txt", "r");

    if (arquivo == NULL) {
        perror("Erro ao abrir o arquivo de entrada");
        return 1;
    }

    char linha[TAMANHO_LINHA];

    while(fgets(linha, TAMANHO_LINHA, arquivo) != NULL){
        printf("%s", linha);
    }
    
    fclose(arquivo);
    return 0;
}

Overwriting leituraArq.c


<h3>Produtores/Consumidores com Arquivos de Texto</h3>

<p>No código a seguir (Código 3) (<a href="https://github.com/jjbaqueta/SO/blob/main/exercicios/produtorConsumidor.c">download do código</a>), é apresentada uma implementação do problema dos produtores e consumidores. Há duas threads produtoras e duas threads consumidoras. Cada produtora lê uma linha de seu respectivo arquivo de texto (um valor inteiro) e envia o dado para o buffer, que é um vetor com duas posições. Os consumidores, por sua vez, retiram os itens do buffer.</p>

<h4 style="color: #2d3436;"><strong>Código 3</strong>: Produtores e consumidores a partir de arquivos de texto.</h4>

In [6]:
%%file produtorConsumidor.c

// Comando de compilação: gcc -Wall produtorConsumidor.c -o out -lpthread
// Comando de execução: ./out

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <semaphore.h>
#include <unistd.h>
#include <string.h>

#define NUMERO_PRODUTORES 2
#define NUMERO_CONSUMIDORES 2
#define TAMANHO_LINHA 50
#define TAMANHO_BUFFER 2
#define ITEM_DE_CONTROLE "TERMINAR"  // Item especial que representa o fim das operações

// --- Semáforos e mutexs ---
sem_t semaforoVagas;
sem_t semaforoItens;
pthread_mutex_t mutexBuffer;

// --- Informação do buffer ---
char buffer[TAMANHO_BUFFER][TAMANHO_LINHA];
int indiceEscrita = 0;
int indiceLeitura = 0;

// Função que implementa os produtores
void* produtor(void* id) {
    long threadId = (long)id;

    // Abrindo o arquivo de texto
    char nomeArquivo[50];
    sprintf(nomeArquivo, "arquivo%ld.txt", threadId + 1);
    FILE *arquivo = fopen(nomeArquivo, "r");

    if (arquivo == NULL) {
        perror("Erro ao abrir o arquivo de entrada");
        pthread_exit(NULL);
    }
    
    // O produtor faz a leitura de cada linha do arquivo    
    char linha[TAMANHO_LINHA];

    while (fgets(linha, TAMANHO_LINHA, arquivo) != NULL) {
        linha[strcspn(linha, "\n")] = 0;    // substitui o '\n' do final linha por um '\0'.

        // Espera por uma vaga no buffer
        sem_wait(&semaforoVagas);

        // Entra na seção crítica para inserir um item no buffer        
        pthread_mutex_lock(&mutexBuffer); 
        strcpy(buffer[indiceEscrita], linha);
        printf("[PRODUTOR %ld] inseriu '%s' no buffer (posição %d)\n", threadId + 1, linha, indiceEscrita);
        indiceEscrita = (indiceEscrita + 1) % TAMANHO_BUFFER;
        pthread_mutex_unlock(&mutexBuffer);
        
        // Avisa que um novo item foi adicionado no buffer
        sem_post(&semaforoItens);

        // Simula trabalho (espera 0.1 segundos)
        usleep(100000);
    }
    printf("[PRODUTOR %ld] terminou sua produção\n", threadId + 1);

    // Fecha o arquivo e finaliza a thread
    fclose(arquivo);
    pthread_exit(NULL);
}

// Função que implementa os consumidores
void* consumidor(void* id) {
    long threadId = (long)id;
    char item[TAMANHO_LINHA];

    while (1) {
        // Espera por um item no buffer
        sem_wait(&semaforoItens);

        // Entra na seção crítica para consumir um item do buffer
        pthread_mutex_lock(&mutexBuffer);
        strcpy(item, buffer[indiceLeitura]);
        printf("[CONSUMIDOR %ld] removeu '%s' do buffer (posição %d)\n", threadId + 1, item, indiceLeitura);
        indiceLeitura = (indiceLeitura + 1) % TAMANHO_BUFFER;
        pthread_mutex_unlock(&mutexBuffer);

        // Avisa que uma posição do buffer está livre
        sem_post(&semaforoVagas);

        // Se o item for o ITEM_DE_CONTROLE, o consumidor é finalizado.
        if (strcmp(item, ITEM_DE_CONTROLE) == 0) {
            printf("[CONSUMIDOR %ld] encontrou o item de controle - Encerrando ...\n", threadId + 1);
            break;
        }
        // Simula trabalho (espera 0.2 segundos)
        usleep(200000);
    }
    pthread_exit(NULL);
}

int main() {
    pthread_t produtores[NUMERO_PRODUTORES];
    pthread_t consumidores[NUMERO_CONSUMIDORES];

    // Rotina para criar arquivos de teste.
    printf("MAIN: Criando arquivos de dados para os produtores...\n");
    
    for(int i = 1; i <= NUMERO_PRODUTORES; i++) {
        char nomeArquivo[50];
        sprintf(nomeArquivo, "arquivo%d.txt", i);
        FILE* arquivo = fopen(nomeArquivo, "w");
        
        fprintf(arquivo, "itemA-%d\n", i);
        fprintf(arquivo, "itemB-%d\n", i);
        fprintf(arquivo, "itemC-%d\n", i);
        fprintf(arquivo, "itemD-%d\n", i);
        fprintf(arquivo, "itemE-%d\n", i);
        fclose(arquivo);
    }
    printf("MAIN: Arquivos criados.\n\n");
    
    // Inicializa os semáforos e mutexes
    sem_init(&semaforoVagas, 0, TAMANHO_BUFFER);
    sem_init(&semaforoItens, 0, 0);
    pthread_mutex_init(&mutexBuffer, NULL);

    // Criando as threads produtoras e consumidoras
    for (long i = 0; i < NUMERO_PRODUTORES; i++) {
        pthread_create(&produtores[i], NULL, produtor, (void*)i);
    }
    for (long i = 0; i < NUMERO_CONSUMIDORES; i++) {
        pthread_create(&consumidores[i], NULL, consumidor, (void*)i);
    }

    // Sincronização: espera pela finalização de todos os produtores
    for (int i = 0; i < NUMERO_PRODUTORES; i++) {
        pthread_join(produtores[i], NULL);
    }

    // Após os produtores terminarem, os consumidores que podem estar dormindo são acordados
    // Utilizando o padrão de design Poison Pill finalizar o processamento dos consumidores
    printf("\nMAIN: Produtores finalizaram. Acordando consumidores...\n");

    for (int i = 0; i < NUMERO_CONSUMIDORES; i++) {

        // Espera uma vaga como um produtor
        sem_wait(&semaforoVagas);

        // Entra na seção crítica para inserir o item terminador no buffer        
        pthread_mutex_lock(&mutexBuffer);
        strcpy(buffer[indiceEscrita], ITEM_DE_CONTROLE);
        indiceEscrita = (indiceEscrita + 1) % TAMANHO_BUFFER;
        pthread_mutex_unlock(&mutexBuffer);

        // Avisa que um novo item foi adicionado no buffer (item de controle)
        sem_post(&semaforoItens);     
    }

    // Sincronização: espera pela finalização de todos os consumidores
    for (int i = 0; i < NUMERO_CONSUMIDORES; i++) {
        pthread_join(consumidores[i], NULL);
    }

    printf("MAIN: Simulação finalizada\n");

    sem_destroy(&semaforoVagas);
    sem_destroy(&semaforoItens);
    pthread_mutex_destroy(&mutexBuffer);
    return 0;
}

Overwriting produtorConsumidor.c
